# SQL query from table names - Continued

In [3]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [4]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [12]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
 CREATE TABLE `account` ( `account_id` integer NOT NULL , `district_id` integer DEFAULT NULL , `frequency` text , `date` integer DEFAULT NULL , PRIMARY KEY (`account_id`) )
 CREATE TABLE `card` ( `card_id` integer DEFAULT NULL , `disp_id` integer DEFAULT NULL , `type` text , `issued` text )
 CREATE TABLE `order` ( `order_id` integer DEFAULT NULL , `account_id` integer DEFAULT NULL , `bank_to` text , `account_to` integer DEFAULT NULL , `amount` float DEFAULT NULL , `k_symbol` text )
 CREATE TABLE `client` ( `client_id` integer DEFAULT NULL , `birth_number` integer DEFAULT NULL , `district_id` integer DEFAULT NULL )
"""} ]



In [6]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [7]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [13]:
#new
context_user = context.copy()
print(return_CCRMSQL("Get the id values of the first 5 clients from district_id with a value equals to 1.", context_user))

```sql
SELECT client_id
FROM client
WHERE district_id = 1
LIMIT 5;
```


In [9]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("give me query for employee who is having maximum salary", old_context_user))

This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query selects the name and salary of the employee with the highest salary by joining the "employees" and "salary" tables on the employee ID. It then orders the results by salary in descending order and limits the output to only the first row, which represents the employee with the maximum salary.


In [14]:
#new
print(return_CCRMSQL("In the client table, get an id value of the last client where the district_id equals to 72.", context_user))

```sql
SELECT client_id
FROM client
WHERE district_id = 72
ORDER BY client_id DESC
LIMIT 1;
```


In [11]:
#old
print(return_CCRMSQL("give me the employee who is not getting salary last month", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
LEFT JOIN salary s ON e.ID_usr = s.ID_usr AND s.year = DATE_SUB(CURDATE(), INTERVAL 1 MONTH)
WHERE s.ID_usr IS NULL;
```

This SQL query retrieves the names of employees who did not receive a salary last month. It uses a LEFT JOIN between the employees and salary tables on the employee ID and filters out the records where the salary for the previous month is NULL, indicating that the employee did not receive a salary.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [16]:
print(return_CCRMSQL("In the order table, which account_ids were responsible for orders between order_id 29540 and order_id 29560 (inclusive)?", context_user))

SELECT account_id
FROM `order`
WHERE order_id BETWEEN 29540 AND 29560;


Learning:
Zero-shot	You just describe the database schema and ask the model to generate SQL — no examples given.
Single-domain	You train (or prompt) the model with examples from the same database or topic.
Cross-domain	You give examples from other databases so the model generalizes to a new one.